# GitHub Data Analysis using Microsoft Fabric Notebook

## Section 1: Setup and Data Loading

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession.builder.getOrCreate()

# Define Azure Data Lake Storage paths
URL = 'abfss://github_event@onelake.dfs.fabric.microsoft.com/github_event_lakehouse.Lakehouse/Tables/'

# Format URLs to access each table's data in the Data Lake Storage
adl_url_events = f"{URL}/public_events/"
adl_url_push_events = f"{URL}/public_push_events/"
adl_url_commits = f"{URL}/public_commits/"
adl_url_pull_request_events = f"{URL}/public_pull_request_events/"
adl_url_trending = f"{URL}/public_trending/"

## Section 2: Load Data from Azure Data Lake Storage

In [ ]:
# Load data from the specified folders in the Azure Data Lake Storage
# The data is stored in Delta format for efficient querying
events_df = spark.read.format("Delta").load(adl_url_events)
push_events_df = spark.read.format("Delta").load(adl_url_push_events)
commits_df = spark.read.format("Delta").load(adl_url_commits)
pull_request_events_df = spark.read.format("Delta").load(adl_url_pull_request_events)
trending_df = spark.read.format("Delta").load(adl_url_trending)

# Create temporary views for SQL queries
events_df.createOrReplaceTempView("events")
push_events_df.createOrReplaceTempView("push_events")
pull_request_events_df.createOrReplaceTempView("pull_requests")
trending_df.createOrReplaceTempView("trending")
commits_df.createOrReplaceTempView("commits")

## Section 3: Data Analysis

###  General GitHub Events Analysis

In [ ]:
# Count the number of events by date to understand general activity trends
activity_df = spark.sql("""
    SELECT date(created_at) as event_date, COUNT(*) as event_count
    FROM events
    GROUP BY event_date
    ORDER BY event_date
""")

# Save the analysis result as a Delta table for further use
activity_df.write.format("delta").mode("overwrite").saveAsTable("activity_count")

### Contributor Activity Analysis

In [ ]:
# Analyze the activity of contributors by counting the number of events each has triggered
contributor_activity_df = spark.sql("""
    SELECT actor_login, COUNT(*) as event_count
    FROM events
    GROUP BY actor_login
    ORDER BY event_count DESC
""")

# Save the analysis result as a Delta table for further use
contributor_activity_df.write.format("delta").mode("overwrite").saveAsTable("contributor_activity")

### Repository Activity Trend Analysis

In [ ]:
# Count the number of push and pull request events per repository to determine the most active ones
repo_activity_df = spark.sql("""
    SELECT events.repo_name,
           COUNT(DISTINCT push_events.id) AS push_count,
           COUNT(DISTINCT pull_requests.id) AS pull_request_count
    FROM events
    LEFT JOIN push_events ON events.id = push_events.event_id
    LEFT JOIN pull_requests ON events.id = pull_requests.event_id
    GROUP BY events.repo_name
    ORDER BY push_count DESC, pull_request_count DESC
""")

# Save the repository activity analysis result for further use
repo_activity_df.write.format("delta").mode("overwrite").saveAsTable("repo_activity")

### Programming Language Popularity Analysis

In [ ]:
# Analyze the popularity of programming languages by counting the number of repositories and summing up the stars
language_popularity_df = spark.sql("""
    SELECT language, COUNT(*) as repo_count, SUM(stars) as total_stars
    FROM trending
    WHERE language IS NOT NULL
    GROUP BY language
    ORDER BY total_stars DESC
""")

# Save the analysis result as a Delta table for further use
language_popularity_df.write.format("delta").mode("overwrite").saveAsTable("language_popularity")

### Commit Analysis by Author

In [ ]:
# Count the number of commits made by each author to understand the most active contributors
commits_by_author_df = spark.sql("""
    SELECT author_name, COUNT(*) as commit_count
    FROM commits
    GROUP BY author_name
    ORDER BY commit_count DESC
""")

# Save the commit analysis result by author for further use
commits_by_author_df.write.format("delta").mode("overwrite").saveAsTable("commits_by_author")

### Repository Popularity Analysis

In [ ]:
# Analyze repository popularity based on the number of stars they received
repo_popularity_df = spark.sql("""
    SELECT repo_name, COUNT(*) as repo_count
    FROM trending
    GROUP BY repo_name
    ORDER BY repo_count DESC
""")
# Save the repository popularity analysis result for further use
repo_popularity_df.write.format("delta").mode("overwrite").saveAsTable("repo_popularity")